In [6]:
from google.colab import drive
drive.mount('/content/drive')
#
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import joblib
#
#----------------------------Definição de Funções------------------------------+
def transform_new(df_new, scaler, encoder, feature_cols):
    """Transforma novos dados usando scaler e encoder já ajustados."""
    df = df_new.copy()

    yes_no_cols = [
        'Col01','Col02','Col03','Col04','Col05','Col06',
        'Col07','Col08','Col09','Col10','Col14','Col16'
    ]
    for col in yes_no_cols:
        df[col] = df[col].map({'Sim': 1, 'Não': 0}).astype('int8')

    df.replace("?", np.nan, inplace=True)
    df['Col12'] = df['Col12'].map({'Masculino': 0, 'Feminino': 1}).astype('int8')

    df[['Col11']] = scaler.transform(df[['Col11']].astype(float))
    df["Col18"] = df.iloc[:, :10].apply(pd.to_numeric, errors='coerce').sum(axis=1)

    categorical_cols = ['Col13', 'Col15', 'Col17']
    df[categorical_cols] = df[categorical_cols].fillna('missing')

    encoded = encoder.transform(df[categorical_cols])
    feature_names = encoder.get_feature_names_out(categorical_cols)

    df = df.drop(columns=categorical_cols).reset_index(drop=True)
    df = pd.concat([df, pd.DataFrame(encoded, columns=feature_names)], axis=1)

    df = df.reindex(columns=feature_cols, fill_value=0)
    return df
#-----------------------------Fim de Funções-----------------------------------+
#
pasta = (
    "/content/drive/MyDrive/PROJETOS/POS-GRADUACAO/[13]-Simone/"
    "Simone-PROFMAT/TEA-ADOSLECENTES/2-PROCESSAMENTO/"
    "TESTE-AGOSTO/"
)

# Carregar preprocessadores e modelo
scaler = joblib.load(pasta + "scaler.pkl")
encoder = joblib.load(pasta + "encoder.pkl")
feature_cols = pd.read_csv(pasta + "feature_cols.csv", header=None).iloc[:,0].tolist()
model = load_model(pasta + "meu_modelo_TEA.h5")

# Arquivo do cliente
arquivo_cliente = "Dados_do_cliente_para_predizer_1x17.xlsx"  # ajuste nome real
df_cliente = pd.read_excel(pasta + arquivo_cliente)

# Transformar
df_cliente_transformado = transform_new(df_cliente, scaler, encoder, feature_cols)
novo_dado = df_cliente_transformado.values.reshape(1, -1)

# Predizer
predicoes = model.predict(novo_dado)
print("\n----- RESULTADOS DA PREDIÇÃO -----")
print(f"RESULTADO: {predicoes[0][0]:.4f}")
print(f"PREDIÇÃO binária: {'TEA' if predicoes[0][0] > 0.5 else 'Não-TEA'}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step

----- RESULTADOS DA PREDIÇÃO -----
RESULTADO: 0.1697
PREDIÇÃO binária: Não-TEA
